# Assignment

## Connecting to Database

In [ ]:
from __future__ import (absolute_import, division, print_function)
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import os
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely import wkt
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = ''
    YOUR_PW     = ''
    DB_LOGIN    = 'y20s1d2x01_'+ YOUR_UNIKEY

    try:
        db_conn_dict = {'host': 'soitpw11d59.shared.sydney.edu.au', 'port': 5432, 'database': DB_LOGIN, 'user':DB_LOGIN , 'password': YOUR_PW }
        conn = psycopg2.connect(**db_conn_dict)
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn

def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None:
                    if msg: print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

In [ ]:
# Connecting to database
conn = pgconnect()

## Task 1: Data Integration and Database Generation

### Creating Tables

In [ ]:
statistical_areas_table = """CREATE TABLE statistical_areas (
                area_id INTEGER PRIMARY KEY,
                area_name VARCHAR(50) , 
                parent_area_id INTEGER
            );"""

neighbourhoods_table = """CREATE TABLE neighbourhoods (
                area_id INTEGER NOT NULL,
                area_name VARCHAR(50),
                land_area NUMERIC,
                population INTEGER,
                number_of_dwellings INTEGER,
                number_of_businesses INTEGER,
                median_annual_household_income INTEGER,
                avg_monthly_rent INTEGER,
                PRIMARY KEY (area_id),
                FOREIGN KEY (area_id) REFERENCES statistical_areas(area_id)
            );"""

census_stats_table = """CREATE TABLE census_stats (
                area_id INTEGER PRIMARY KEY, 
                median_annual_household_income INTEGER,
                avg_monthly_rent INTEGER
            );"""

population_stats_2016_table = """CREATE TABLE population_stats_2016 (
                area_id INTEGER PRIMARY KEY,
                area_name VARCHAR(50),
                age_0_to_4 INTEGER,
                age_5_to_9 INTEGER,
                age_10_to_14 INTEGER,
                age_15_to_19 INTEGER,
                age_20_to_24 INTEGER,
                age_25_to_29 INTEGER,
                age_30_to_34 INTEGER,
                age_35_to_39 INTEGER,
                age_40_to_44 INTEGER,
                age_45_to_49 INTEGER,
                age_50_to_54 INTEGER,
                age_55_to_59 INTEGER,
                age_60_to_64 INTEGER,
                age_65_to_69 INTEGER,
                age_70_to_74 INTEGER,
                age_75_to_79 INTEGER,
                age_80_to_84 INTEGER,
                age_85_and_over INTEGER,
                total_persons INTEGER,
                females INTEGER,
                males INTEGER
            );"""

health_services_table = """CREATE TABLE health_services (
                id INTEGER PRIMARY KEY,
                name VARCHAR(100),
                category VARCHAR(50),
                num_beds INTEGER,
                address VARCHAR(100),
                suburb VARCHAR(50),
                state VARCHAR(4),
                postcode CHAR(4),
                longitude NUMERIC,
                latitude NUMERIC,
                comment VARCHAR(2300),
                website VARCHAR(150)
            );"""

nsw_postcodes_table = """CREATE TABLE nsw_postcodes (
                id INTEGER PRIMARY KEY,
                postcode INTEGER,
                locality VARCHAR(50),
                longitude NUMERIC,
                latitude NUMERIC
            );"""

nsw_testsites_table = """CREATE TABLE nsw_testsites (
                site_id INTEGER PRIMARY KEY,
                Centre_name VARCHAR(70) ,
                Phone_number VARCHAR(70),
                Opening_hours VARCHAR(80),
                longitude NUMERIC,
                latitude NUMERIC,
                test_capacity INTEGER
            );"""

covid_tests_table = """CREATE TABLE covid_tests (
                test_id INTEGER PRIMARY KEY,
                test_date DATE,
                postcode CHAR(4),
                lhd_2010_code CHAR(4),
                lhd_2010_name VARCHAR(50),
                lga_code19 INTEGER,
                lga_name19 VARCHAR(50),
                result VARCHAR(50)
            );"""

SA2_data_table = """CREATE TABLE sa2_data (
                id INTEGER PRIMARY KEY,
                name VARCHAR(50),
                geom GEOMETRY(MULTIPOLYGON,4326)
            );"""

# DO NOT EXECUTE THIS CODE - the tables have already been created in the database
# pgquery(conn, "SET search_path to covid")
# pgquery(conn, statistical_areas_table)
# pgquery(conn, neighbourhoods_table)
# pgquery(conn, census_stats_table)
# pgquery(conn, population_stats_2016_table)
# pgquery(conn, health_services_table)
# pgquery(conn, nsw_testsites_table)
# pgquery(conn, covid_tests_table)

# We need a reference to the public schema in order to use PostGIS
# pgquery(conn, "SET search_path to covid, public")
# pgquery(conn, SA2_data_table)

### Loading data and inserting them into tables

In [ ]:
# DO NOT EXECUTE THIS CODE - the data has already been loaded into the database

# conn.execute("SET search_path to covid")


## Loading data into statistical_areas
# statistical_areas_data = pd.read_csv("datasets/StatisticalAreas.csv")
# for index, row in statistical_areas_data.iterrows():
#     conn.execute("INSERT INTO statistical_areas VALUES (%s, %s, %s)", (row[0], row[1], row[2]))


# # Loading data into neighbourhoods
# neighbourhoods_data = pd.read_csv("datasets/Neighbourhoods.csv")
# for index, row in neighbourhoods_data.iterrows():
#     # sorry about this
#     if str(row[2]) == "nan":
#         row[2] = None
#     if str(row[3]) == "nan":
#         row[3] = None
#     if str(row[4]) == "nan":
#         row[4] = None
#     if str(row[5]) == "nan":
#         row[5] = None
#     if str(row[6]) == "nan":
#         row[6] = None
#     if str(row[7]) == "nan":
#         row[7] = None
#     conn.execute("INSERT INTO neighbourhoods VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]))




# #Loading data into nsw_testsites
# covid19_nsw_testsites_simulated_capacity_data = pd.read_csv("datasets/covid19_nsw_testsites_simulated_capacity.csv")
# for index, row in covid19_nsw_testsites_simulated_capacity_data.iterrows():
#     conn.execute("INSERT INTO nsw_testsites VALUES (%s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6]))

# # Loading data into nsw_postcodes
# nsw_postcodes_data = pd.read_csv("datasets/NSW_Postcodes.csv")
# for index, row in nsw_postcodes_data.iterrows():
#     conn.execute("INSERT INTO nsw_postcodes VALUES (%s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4]))

# # # Loading data into population_stats_2016
# population_stats_2016_data = pd.read_csv("datasets/PopulationStats2016.csv")
# for index, row in population_stats_2016_data.iterrows():
#     # oh my god
#     row_tuple = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[21], row[22])
#     conn.execute("INSERT INTO population_stats_2016 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row_tuple)


# # Loading data into census_stats
# census_stats_data = pd.read_csv("datasets/CensusStats.csv")
# for index, row in census_stats_data.iterrows():
#     # Numpy data types don't like being converted to None
#     if str(row[1]) == "nan" and str(row[2]) != "nan":
#         conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], None, row[2]))
#         continue
#     if str(row[2]) == "nan" and str(row[1]) != "nan":
#         conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], row[1], None))
#         continue
#     if str(row[2]) == "nan" and str(row[1]) == "nan":
#         conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], None, None))
#         continue
#     conn.execute("INSERT INTO census_stats VALUES (%s, %s, %s)", (row[0], row[1], row[2]))

# Loading data into health_services
# health_services_data = pd.read_csv("datasets/HealthServices.csv")
# for index, row in health_services_data.iterrows():
#     if str(row[3]) == "nan":
#         row[3] = None
#     conn.execute("INSERT INTO health_services VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10]))

# # Loading data into covid_tests
# with open("datasets/tests_by_date_location_result.json", "r") as f:
#     covid_test_data = json.load(f)
# for record in covid_test_data['records']:
#     conn.execute("INSERT INTO covid_tests VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (record[0], record[1], record[2], record[3], record[4], record[5], record[6], record[7]))

# Loading spatial data in sa2

# geom_wkt_data = []
# for index, row in SA2_data.iterrows():
#     try:
#         geom_wkt_data.append(row['geometry'].wkt) # converting geometry polygon data to a string format
#     except:
#         geom_wkt_data.append(None)
# SA2_data = SA2_data.assign(geom_wkt = geom_wkt_data) # adding string formatted geometry to dataframe
# conn.execute("SET search_path to covid, public")
# for index, row in SA2_data.iterrows():
#     conn.execute("INSERT INTO sa2_data VALUES (%s, %s, ST_Multi(ST_GeomFromText(%s, 4326)))", (row[0], row[2], row[13]))

## Add in geometry columns for health_services, NSW_Postcodes & Neighbourhoods

In [ ]:
# pgquery(conn, "SET search_path to covid, public")

# Adding geom column to neighbourhoods
# pgquery(conn, "ALTER TABLE health_services ADD geom GEOMETRY(POINT, 4326)")
# for i in range(3717):
#     query = """UPDATE health_services SET geom = 
#         (SELECT ST_SetSRID(ST_Point(CAST(longitude as float), CAST(latitude as float)), 4326) FROM health_services WHERE id = {})
#         WHERE id = {}
#         """.format(i, i)
#     pgquery(conn, query)
    
# Adding geom column to postcodes    
# pgquery(conn, "ALTER TABLE nsw_postcodes ADD geom GEOMETRY(POINT, 4326)")
# for i in range(22873):
#     query = """UPDATE nsw_postcodes SET geom = 
#         (SELECT ST_SetSRID(ST_Point(CAST(longitude as float), CAST(latitude as float)), 4326) FROM nsw_postcodes WHERE id = {})
#         WHERE id = {}
#         """.format(i, i)
#     pgquery(conn, query)

# Adding geom column to neighbourhoods
# pgquery(conn, "ALTER TABLE neighbourhoods ADD geom GEOMETRY(MULTIPOLYGON, 4326)")
# res = pgquery(conn, "SELECT area_id FROM neighbourhoods")
# for index in res:
#     query = """UPDATE neighbourhoods SET geom = 
#          (SELECT geom FROM sa2_data WHERE id = {})
#          WHERE area_id = {}
#          """.format(index[0], index[0])
#     pgquery(conn, query)

## Closing Connection

In [ ]:
conn.close()
db.dispose()